# Normalize Deaths by mobility

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys; sys.path.append('../')
from src.data_loader.data_loader import load_google_mobility, load_deaths, load_interventions, load_counties, load_google_mobility_time_series
from src.utils.dates import get_today
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Time series data
mobility, mobility_date = load_google_mobility()
deaths, deaths_date = load_deaths(join_county_codes=False)
interventions, interventions_date = load_interventions()

# Static data
counties, counties_date = load_counties()

# Processed mobility -> time series
mobility_ts, mobility_ts_date = load_google_mobility_time_series()

In [3]:
mobility_ts

,Unnamed: 0,FIPS,state,county,02-15,02-16,02-17,02-18,02-19,02-20,...,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11
0,0,1001,AL,Autauga County,5.0,0.0,8.0,-2.0,-2.0,-8.0,...,-24.0,-24.0,-39.0,-53.0,-30.0,-35.0,-31.0,-29.0,-30.0,-37.0
1,1,1003,AL,Baldwin County,17.0,8.0,9.0,7.0,12.0,5.0,...,-40.0,-39.0,-48.0,-55.0,-46.0,-47.0,-42.0,-41.0,-41.0,-44.0
2,2,1005,AL,Barbour County,5.0,-3.0,12.0,4.0,1.0,-6.0,...,-10.0,-9.0,-22.0,-40.0,NaN,NaN,NaN,NaN,-18.0,NaN
3,3,1007,AL,Bibb County,14.0,-3.0,7.0,-4.0,-1.0,-1.0,...,-8.0,-10.0,-12.0,-42.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1009,AL,Blount County,7.0,1.0,5.0,-8.0,2.0,-6.0,...,-23.0,-20.0,-23.0,-49.0,NaN,NaN,NaN,NaN,-25.0,-20.0
5,5,1011,AL,Bullock County,NaN,NaN,-40.0,NaN,NaN,NaN,...,NaN,-22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,1013,AL,Butler County,2.0,2.0,1.0,0.0,2.0,-6.0,...,-23.0,-27.0,-41.0,-47.0,NaN,NaN,NaN,NaN,NaN,NaN
7,7,1015,AL,Calhoun County,7.0,1.0,2.0,-2.0,0.0,-7.0,...,-29.0,-29.0,-41.0,-53.0,-39.0,-36.0,-37.0,-32.0,-35.0,-40.0
8,8,1017,AL,Chambers County,2.0,-4.0,2.0,-3.0,2.0,0.0,...,-18.0,-26.0,-39.0,-46.0,-30.0,-33.0,-37.0,-24.0,-31.0,-35.0
9,9,1019,AL,Cherokee County,4.0,-2.0,-8.0,-12.0,-2.0,-10.0,...,-16.0,-10.0,-23.0,-41.0,NaN,NaN,NaN,NaN,NaN,NaN
